In [1]:
import yfinance as yf
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import joblib

In [2]:
nifty50_monthly = yf.Ticker('^NSEI')
nifty50_monthly = nifty50_monthly.history(start='2014-01-01', end='2024-1-1', interval='1mo')

nifty50_monthly["Tommorow"] = nifty50_monthly['Close'].shift(-1)
nifty50_monthly["Monthly_Price_Change"] = nifty50_monthly["Tommorow"] - nifty50_monthly["Close"]
nifty50_monthly['Target'] = (nifty50_monthly['Tommorow'] > nifty50_monthly['Close']).astype(int)

nifty50_monthly = nifty50_monthly.dropna()
del nifty50_monthly["Dividends"]
del nifty50_monthly["Stock Splits"]
train_month = nifty50_monthly.iloc[:-12]
test_month = nifty50_monthly.iloc[-12:] 

In [61]:
Predictors_month = ['Close', 'Volume', 'Open', 'High', 'Low']
regressor_month =  RandomForestRegressor(min_samples_split= 78, n_estimators= 100 )
regressor_month.fit(train_month[Predictors_month], train_month["Monthly_Price_Change"])

monthly_price_change_preds = regressor_month.predict(test_month[Predictors_month])

rmse_month = root_mean_squared_error(test_month["Monthly_Price_Change"], monthly_price_change_preds)
print(f"Mean Squared Error: {rmse_month}")

Mean Squared Error: 675.3893401004659


In [62]:
joblib.dump(regressor_month, 'monthly_price_change_predictor.pkl')

['monthly_price_change_predictor.pkl']

In [4]:
nifty50_monthly

,Open,High,Low,Close,Volume,Tommorow,Monthly_Price_Change,Target
Date,,,,,,,,
2014-01-01 00:00:00+05:30,6301.250000,6358.299805,6027.250000,6089.500000,3280800,6276.950195,187.450195,1
2014-02-01 00:00:00+05:30,6058.799805,6282.700195,5933.299805,6276.950195,2688200,6704.200195,427.250000,1
2014-03-01 00:00:00+05:30,6264.350098,6730.049805,6212.250000,6704.200195,3853200,6696.399902,-7.800293,0
2014-04-01 00:00:00+05:30,6729.500000,6869.850098,6650.399902,6696.399902,2672300,7229.950195,533.550293,1
2014-05-01 00:00:00+05:30,6709.950195,7563.500000,6638.549805,7229.950195,4282600,7611.350098,381.399902,1
...,...,...,...,...,...,...,...,...
2023-07-01 00:00:00+05:30,19246.500000,19991.849609,19234.400391,19753.800781,5802500,19253.800781,-500.000000,0
2023-08-01 00:00:00+05:30,19784.000000,19795.599609,19223.650391,19253.800781,6027500,19638.300781,384.500000,1
2023-09-01 00:00:00+05:30,19258.150391,20222.449219,19255.699219,19638.300781,5666500,19079.599609,-558.701172,0


Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000, min_samples_split=67, random_state=1)

Predictors = ['Close', 'Volume', 'Open', 'High', 'Low']
model.fit(train_month[Predictors], train_month["Target"])

preds = pd.Series(model.predict(test_month[Predictors]))

from sklearn.metrics import precision_score
precision_score(test_month['Target'], preds)

1.0

In [24]:
joblib.dump(model, 'monthly_classifier.pkl')

['monthly_classifier.pkl']